# Restaurant Recommendation Model

## Purpose
This Python notebook is dedicated to building and training the machine learning model used as the backend for the Restaurant Recommendation System. The goal is to create a robust recommendation system that suggests restaurants based on user preferences, historical data, and other relevant features.

## Frameworks Used
Built with sci.kit learn and associated libraries. The cosine similarity vector model was used to build this system.

In [1]:
#Import Relevant Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Read Pandas Dataframe
complete = pd.read_csv("./data/restaurantdemo.csv") #Directory
complete = complete.drop(columns=['business_id', 'GEO_ID', 'address'])
complete = complete.head(20000)

print(complete.head())

                       name         city state  postal_code   latitude  \
0              Metro Grille     Flanders    NJ         7836  39.949904   
1  Helen's Cafe and Gardens      Alloway    NJ         8001  39.563830   
2       Alloway Village Inn      Alloway    NJ         8001  39.555717   
3                McDonald's  Cherry Hill    NJ         8002  39.936079   
4              Little Tokyo  Cherry Hill    NJ         8002  39.943728   

   longitude  stars  review_count  \
0 -75.161599    3.0            17   
1 -75.363824    4.0             5   
2 -75.360766    3.5            13   
3 -75.044117    2.5            18   
4 -75.026066    4.0            12   

                                          categories  B01001_001E  ...  \
0          Restaurants, Asian Fusion, American (New)        12034  ...   
1  Restaurants, Cafes, Breakfast & Brunch, Venues...          835  ...   
2  American (Traditional), Restaurants, Bars, Nig...          835  ...   
3  Burgers, Food, Restaurants, Fast 

/var/folders/6l/xb2bmr4d5718m4gjtrxd8h2w0000gn/T/ipykernel_34115/1506284659.py:2: DtypeWarning: Columns (116,117,123,124,125,127,128,129,130,132,133,139,140,145,146,147,149,150,153,154) have mixed types. Specify dtype option on import or set low_memory=False.
  complete = pd.read_csv("./data/restaurantdemo.csv") #Directory


In [11]:
#Sk.Learn Imports
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder

In [12]:
#Boosting Regressor Sk.Learn
y = complete['stars']
X = complete.drop(columns=['star_count', 'stars'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Text Columns
text_features = ['name', 'city', 'categories', 'address']

numeric_features = X_train.select_dtypes(include=['number']).columns

#TF-IDF
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_features),
        ('name', TfidfVectorizer(), 'name'),
        ('city', TfidfVectorizer(), 'city'),
        ('categories', TfidfVectorizer(), 'categories'),
    ])

gb_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)

model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', gb_model)])

#Train
model.fit(X_train, y_train)

#Evaluate
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 0.43657840168681056


## BERT Shenanigans

In [3]:
from transformers import BertTokenizer, BertModel
import torch

/Users/mihir/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/mihir/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [5]:
# Text Columns
text_features = ['name', 'city', 'categories']
bert_embeddings_list = []

for column in text_features:
    tokenized = tokenizer(complete[column].tolist(), return_tensors='pt', padding=True, truncation=True)

    with torch.no_grad():
        outputs = model(**tokenized)

    embeddings = outputs.last_hidden_state[:, 0, :]
    bert_embeddings_list.append(embeddings.numpy())

concatenated_embeddings = np.concatenate(bert_embeddings_list, axis=1)

#New Columns
new_column_names = [f'{col}_bert_embedding_{i}' for col in text_features for i in range(embeddings.shape[1])]

#New DF
bert_embeddings_df = pd.DataFrame(concatenated_embeddings, columns=new_column_names)

df = pd.concat([complete, bert_embeddings_df], axis=1)

: 